In [1]:
import os
import time
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd
from pandas_gbq import to_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

from module.bigquery import BigqueryProcessor
from module.slack import SlackMessageRetriever


In [2]:
datetime.today().strftime("%Y-%m-%d")

'2023-05-21'

In [3]:
# 인스턴스 생성
bigquery_client = BigqueryProcessor(
    env_name="GOOGLE_APPLICATION_CREDENTIALS", database_id="geultto_8th"
)
slack_app = SlackMessageRetriever(env_name="SLACK_TOKEN")

In [4]:
where_clause = "tddate >= '2023-01-24' and tddate <= '2023-05-16'"
df = bigquery_client.read_table(table_name='slack_conversation', where_clause=where_clause)

In [5]:
len(df.loc[(df.message_type == 'post'), 'createtime'].unique())

5179

In [19]:
df.iloc[70, ]

channel_id                                             C04LMFQ95DM
message_type                                                thread
post_id                     U04L5NJLUVB-2023-02-20-10-12-42-373849
user_id                                                U04KU62DJTZ
createtime                              2023-02-20 10:46:49.872399
tddate                                                  2023-02-20
text                  이해했습니다! 설명해주셔서 감사합니다 :slightly_smiling_face:
reactions        [{"name": "raised_hands", "user_id": ["U04L5NJ...
createtime_v2                     2023-02-20 01:46:49.872399+00:00
unixtime_1                                       1676890009.872399
unixtime_2                                       1676857609.872399
Name: 70, dtype: object

In [5]:
df['createtime_v2'] = df['createtime'].dt.tz_localize('Asia/Seoul').dt.tz_convert('UTC')
df['unixtime_1'] = df['createtime'].astype('int64') / 1e9
df['unixtime_2'] = df['createtime_v2'].astype('int64') / 1e9

In [8]:
time.mktime(datetime(2020, 1, 1).timetuple())

1577804400.0

In [8]:
print(df['createtime'][70])
print(df['unixtime_1'][70])
print(df['createtime_v2'][70])
print(df['unixtime_2'][70])
print(datetime.fromtimestamp(1676890009.872399))
print(datetime.fromtimestamp(1676857609.872399))

2023-02-20 10:46:49.872399
1676890009.872399
2023-02-20 01:46:49.872399+00:00
1676857609.872399
2023-02-20 19:46:49.872399
2023-02-20 10:46:49.872399


In [49]:
df.loc[(df.post_id == 'U04LRNDS67J-2023-04-25-20-41-03-550789') & (df.message_type == 'post'), ]['unixtime'].tolist()[0]

1682422863.550789

In [50]:
print(datetime.fromtimestamp(1682422863.550789))

2023-04-25 20:41:03.550789


In [51]:
slack_app = SlackMessageRetriever(env_name="SLACK_TOKEN")

In [53]:
datetime(2023, 4, 25).timestamp()

1682348400.0

In [60]:
datetime(2023, 4, 26).timestamp()

1682434800.0

In [16]:
posts = slack_app.read_post_from_slack(
    channel_id="C04KMJ40TGX",
    start_unixtime=1682348400.0,
    end_unixtime=1682434800.0,
)

In [28]:
temp_df = pd.DataFrame()
temp_df.empty

True

In [27]:
for i in range(2):
    try:
        slack_app.read_thread_from_slack(channel_id="C04KMJ40TGX", thread_ts=1682422863.550799)
    except Exception as e:
        print(e)

The request to the Slack API failed. (url: https://www.slack.com/api/conversations.replies)
The server responded with: {'ok': False, 'error': 'thread_not_found'}
The request to the Slack API failed. (url: https://www.slack.com/api/conversations.replies)
The server responded with: {'ok': False, 'error': 'thread_not_found'}


In [11]:
df.loc[df.text.str.contains('scrolling 이 UX 에 어떻게 도움이 되는지'), 'unixtime_2'].tolist()[0]

1684074864.147899

In [6]:
df.loc[(df.unixtime_2 == 1682422863.550789), ]

,channel_id,message_type,post_id,user_id,createtime,tddate,text,reactions,createtime_v2,unixtime_1,unixtime_2
6217,C04KMJ40TGX,post,U04LRNDS67J-2023-04-25-20-41-03-550789,U04LRNDS67J,2023-04-25 20:41:03.550789,2023-04-25,<!channel>\n안녕하세요 :slightly_smiling_face: 이벤트...,"[{""name"": ""raised_hands"", ""user_id"": [""U04LRND...",2023-04-25 11:41:03.550789+00:00,1.682455e+09,1.682423e+09


In [12]:
response = slack_app.read_thread_from_slack(channel_id="C04M0FZSZR9", thread_ts=1684074864.147899)

In [13]:
response

[{'bot_id': 'B04LSG8ADCG',
  'type': 'message',
  'text': "\n&gt;&gt;&gt;:tada: *<@U04LM7AQ8QZ>님 제출 완료.*                \n\n:speech_balloon: 'SwiftUI 에서 programmatic scrolling 이 UX 에 어떻게 도움이 되는지 직접 경험해보고, 코드를 통해 anchor 파라미터의 역할을 확인해봅니다.'\n                \ncategory : 기술 &amp; 언어                \ntag : `SwiftUI` `iOS`                \nlink : <https://medium.com/@Jager-yoo/swiftui-%EC%BD%94%EB%93%9C%EB%A5%BC-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%81%AC%EB%A1%A4%EB%A7%81%EC%9D%84-%EC%9E%91%EB%8F%99%EC%8B%9C%ED%82%A4%EB%8A%94-%EB%B0%A9%EB%B2%95-programmatic-scrolling-c567c99eb31f>",
  'user': 'U04L59HJMM2',
  'ts': '1684074864.147899',
  'app_id': 'A04KN89JSKH',
  'blocks': [{'type': 'rich_text',
    'block_id': 'iU==c',
    'elements': [{'type': 'rich_text_quote',
      'elements': [{'type': 'emoji', 'name': 'tada', 'unicode': '1f389'},
       {'type': 'user', 'user_id': 'U04LM7AQ8QZ', 'style': {'bold': True}},
       {'type': 'text', 'text': '님 제출 완료.', 'style': {'bold': True}},
       {'type':

In [18]:
response[6]

{'type': 'message',
 'subtype': 'thread_broadcast',
 'text': '저도 받고 싶습니다!! 감사합니다!!',
 'user': 'U04LYJ4LQQY',
 'ts': '1682422921.977219',
 'thread_ts': '1682422863.550789',
 'root': {'client_msg_id': '680ee1e5-ab05-4493-9207-64041df010c2',
  'type': 'message',
  'text': '<!channel>\n안녕하세요 :slightly_smiling_face:  이벤트라 채널을 해봅니다!\n*인프런과 협업하면서 글또분들을 위한 30% 쿠폰을 받아왔어요-!*\n• 할인율 : 30%\n• 쿠폰 사용 기간 : ~ 23년 7월 31일\n• 1회만 사용이 가능한 쿠폰이며 다른 할인과 중복은 어렵고, 프로모션 비동의 강의는 제외됩니다\n*쿠폰을 받고 싶으신 분은 쓰레드에 "댓글"을 남겨주세요. 한분 한분씩 제가 DM으로 쿠폰 코드를 보내드릴게요(실시간으로 바로는 아니여도 하루에 1번씩 보낼게요)*\n\n그리고 *그 외에 강의를 만들고 싶으신 분이 있다면 이모지(*:books:*) 남겨주셔요*- 얼마나 되는지 확인차 남겨보아요(다만 7기에서 강의 만들기 모임을 했을 때 결국 만드신 분은 1분 뿐이였어요..! 시간 확보를 충분히 할 수 있는 분에게 추천드려요) -&gt; 강의 희망자는 :books: 이모지 남겨주세용. 추후에 제가 별도로 채널을 만들지 기준으로 해볼게요(궁금하신거 질문하셔도 괜찮아요)',
  'user': 'U04LRNDS67J',
  'ts': '1682422863.550789',
  'blocks': [{'type': 'rich_text',
    'block_id': 'WSyg',
    'elements': [{'type': 'rich_text_section',
      'elements': [{'type': 'broadcast', 'range': 'cha

In [61]:
len(response)

151

In [65]:
empty_list = []
for thread in response:
    empty_list.append(
        SlackMessageRetriever.convert_thread_to_dict(
            channel_id="C04KMJ40TGX", thread=thread
        ))

In [66]:
pd.DataFrame(empty_list)

,channel_id,message_type,post_id,user_id,createtime,tddate,text,reactions
0,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04LS2B38V7,2023-04-25T20:41:27.570899,2023-04-25,저요!! 감사합니다~,[]
1,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04M43NV15E,2023-04-25T20:41:52.096349,2023-04-25,와우 ㅠ 저두용! 감사합니다:),[]
2,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04L8HEQ11Q,2023-04-25T20:41:54.999759,2023-04-25,와…! 저 받고싶습니다~! 이런 기회 마련해주셔서 감사합니다. :man-bowing:,[]
3,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04M3AL9M08,2023-04-25T20:41:56.108649,2023-04-25,저도 받고싶습니다! 감사합니다!,[]
4,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04LT3TGEBT,2023-04-25T20:41:57.377959,2023-04-25,저두요 !!,[]
...,...,...,...,...,...,...,...,...
146,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04KU1V44T1,2023-05-01T14:34:23.900079,2023-05-01,남았을까요...?ㅜㅜ 저도 신청해 봅니다!!,[]
147,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04KMKH8UR5,2023-05-02T01:37:11.740079,2023-05-02,"변예진님, 소재훈님, 이지원님, 박이슬님, 강수경님 발송드렸습니다.","[{""name"": ""raised_hands"", ""user_id"": [""U04LRND..."
148,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04L3ALEKDL,2023-05-02T02:07:08.658229,2023-05-02,저도 신청합니다. 감사합니다,[]
149,C04KMJ40TGX,thread,U04LRNDS67J-2023-04-25-20-41-03-550789,U04M02MJAKT,2023-05-13T10:13:14.332579,2023-05-13,호옥시 아직 신청할 수 있다면 저도 부탁드립니다!,[]


In [20]:
def getDateList_byday(start_date : str, end_date : str, day : int) -> list:
 
    date_list = []
    convert_startDate = datetime.strptime(start_date, "%Y-%m-%d").date()
 
    start = convert_startDate + relativedelta(days=-1)
 
    while start.strftime('%Y-%m-%d') < end_date:
        start = start + relativedelta(days=+1)
        if start.weekday() == day:
            date_list.append(start)
 
    return date_list

In [21]:
getDateList_byday(start_date = '2023-05-01', end_date = '2023-05-30', day=6)

[datetime.date(2023, 5, 7),
 datetime.date(2023, 5, 14),
 datetime.date(2023, 5, 21),
 datetime.date(2023, 5, 28)]

In [ ]:
slack_app = conversations_replies(channel=channel_id, ts=thread_ts)

In [4]:
import sys
from datetime import datetime

# 모듈 참조 경로 추가
sys.path.append("/Users/cho/git/geultto_genie_bot")

from module.bigquery import BigqueryProcessor
from module.slack import SlackMessageRetriever

# 인스턴스 생성
bigquery_client = BigqueryProcessor(
    env_name="GOOGLE_APPLICATION_CREDENTIALS", database_id="geultto_8th"
)
schema_conversation = bigquery_client.read_schema(file_path="schema/slack_conversation.json")
slack_app = SlackMessageRetriever(env_name="SLACK_TOKEN")

# post_id 읽어오기
sdate = datetime.today().strftime("%Y-%m-%d")
where_clause = f"tddate <= '{sdate}'"
df_conversation = bigquery_client.read_table(
    table_name="slack_conversation", where_clause=where_clause
)
print(df_conversation["tddate"].max())

2023-05-21


In [3]:
df_conversation.shape

(16833, 8)

In [5]:
bigquery_client.update_table(
    df=df_conversation, table_name='upsert_test', if_exists="replace", schema=None
)

geultto.geultto_8th.upsert_test가 정상적으로 적재 됐습니다.
